In [21]:
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [22]:
# doc_folder = "../base_info"

In [23]:
# Function to embed chunks and create vector store
def embed_chunks(chunks):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512)
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store


In [24]:
from glob import glob
import os

In [25]:
tool_files =  ["tools.txt"]

In [26]:
def getQaChunks(qa_file):
    with open(qa_file, 'r') as f:
        qas_data = f.read()
    qas = qas_data.split('###')
    qas = [i.strip() for i in qas if len(i.strip()) > 0]
    return qas

In [27]:
all_tools = []
for tool_file in tool_files:
    file_tools = getQaChunks(tool_file)
    all_tools += file_tools

In [28]:
len(all_tools)

33

In [29]:
lengths = []
for tool in all_tools:
    lengths.append(len(tool)/4)

In [37]:
sum(lengths)*10000*4*2.5*0.15*85/1000000

1949.79375

In [38]:
vstore = embed_chunks(all_tools)

In [24]:
vstore.similarity_search("when to sow my new grape seeds?")

[Document(page_content="Q: What is the right time to sow the seeds?\nA: Check the weather forecast for the next few days (tool call), and then decide the right time for the crop, based on own's knowledge."),
 Document(page_content='Q: What should be the optimum depth of seed to be sown?\nA: Seed to be sown at the depth of twice the seed size'),
 Document(page_content='Q: When should we take the petiole and soil samples?\nA: It varies crop-wise.'),
 Document(page_content='Q: What should be the optimum rainfall for sowing?\nA: 50 to 100 mm is optimum rain fall required for sowing')]

In [39]:
vstore.save_local("vstore", index_name="tools")

In [40]:
store = FAISS.load_local("vstore", index_name="tools", embeddings=OpenAIEmbeddings(model="text-embedding-3-small", dimensions=512), allow_dangerous_deserialization=True)

In [41]:
store.similarity_search_with_score("get fertigation schedule")

[(Document(id='cebc6f03-a872-4c18-adc8-702c71461dc1', metadata={}, page_content='get_fertigation_schedule  \nChecks the farmer\'s crop stages and requirements with internal petiole reports and gives the current week fertigation schedule details, and details of the next fertigation schedule (could be next week in the same crop stage or in next crop stage) for the plots. Fertilizers are given by 3 distribution methods:  \n- Fertigation: fertilizers put into the irrigation pipeline itself  \n- Foliar: on the leaves using a foliar spray  \n- Soil: directly on the soil  \nIf the unit of fertilizer amount includes "per acre", then it is for 1 acre. Otherwise it is for "units per 1 plant".'),
  0.56048083),
 (Document(id='2b28168a-aac6-48f0-8fc8-48ca941ce302', metadata={}, page_content='get_specific_fertigation_requirements  \nFinds the fertigation requirements for the plots with a particular crop stage and week numbers.'),
  0.74425715),
 (Document(id='d2b7287e-007c-4cfe-8ee6-c99cc3cc97c3', 